In [49]:
from variables import *
from SDPC_utils import *

import ctypes
import pickle as pkl
import pandas as dd
import numpy as np
import cvxpy as cp
from collections import OrderedDict
from time import time, strftime
from os import getcwd
import re
import warnings
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from scipy.sparse import csr_matrix, save_npz, load_npz, diags
from scipy.sparse.linalg import lsqr
from scipy.linalg import orth

warnings.filterwarnings("ignore")

In [64]:
def cargar_data(query=None, to_sql = False):
    """Toma los datos del .lst indicado o hace query a SQL server. 
    Genera Maestro e Historico."""
    
    t0 = time()
    if type(query) == type(None):
        lsts = [p for p in os.listdir(TMP_PATH) if p.endswith('.lst')]
        print(f"{len(lsts)} '.lst's encontrados.")
        df = pd.DataFrame()
        for p in lsts:
            df_ = read_lst(os.path.join(TMP_PATH, p), lst_cols)
            df_ = preprocesamiento(df_)
            df_['Base'] = p.split('.')[0]
            df = pd.concat([df, df_])       
        
        df.groupby([cl_col, s_col, p_col]) \
            .agg({neto_col: 'mean'}) \
            .sort_index()[neto_col].reset_index().to_csv(TMP_PATH+'Historico.csv', sep=';', index = False)
                
        #Exportar maestr
        wb = xw.Book(MASTER_PATH)
        wb.sheets['Maestro'].range('A2', 'T200000').clear()
        wb.sheets['Maestro'].range('A6').value = ''
        wb.sheets['Maestro'].range('A1').options(pd.DataFrame, index=False).value = df.groupby([cl_col, s_col]).agg({c_col:'last', nom_cli_col: 'last', 
                                                                                                                   cir_col:'last'}).reset_index()
        
    else:
        engine = sqlalchemy.create_engine(f"mssql+pyodbc://{UID}:{PWD}@{SERVER}/{DATABASE}?driver=SQL+Server")
        df = pd.read_sql_query(query, engine, index_col='index').to_csv(TMP_PATH+'Historico.csv', sep=';', index = False)
    
    if to_sql:
        to_sql(df)
   
    ctypes.windll.user32.MessageBoxW(0, f"Datos cargados. {time() - t0:.0f} segundos.",  "Terminado", 1)

In [51]:
def preprocesamiento(df):
    from variables import c_col, cl_col, p_col, z_col, s_col, fl_col, fle_col, cir_col, q_col, dev_col, neto_col
    
    # Tranformo a numerico
    df[[q_col, dev_col]] = df[[q_col, dev_col]].apply(lambda x: pd.to_numeric(x.astype(str).str.replace(',','.'), errors='coerce'))
    df[[c_col, cl_col, p_col, z_col, s_col, fl_col, fle_col, cir_col]] = df[[c_col, cl_col, p_col, z_col, s_col, fl_col, fle_col, cir_col]].apply(lambda x: pd.to_numeric(x, errors= 'coerce', downcast='integer'))
    
    # Saco nulos y el ultimo que era '\x00\x00\x00\x00\x00\x00\x00\x00'
    df = df.dropna(how='all')
    #df = df[:-1]
    
    df = df.rename({'Regi¢n':'Region'}, axis=1)

    # Calculo venta bruta 
    df[neto_col] = (df[q_col] - df[dev_col]).astype('float64')

    # Se borran ventas netas negativas
    df = df[df[neto_col] > 0]
    
    df = df.groupby([cl_col, s_col, p_col])\
        .agg({c_col:'last', z_col:'last', nom_cli_col: 'last', cir_col:'last', fl_col:'last', neto_col:'sum'}) \
        .reset_index()
    
    return df

In [52]:
def distribuir_como(df):
    """Copia distribución entre productos.
    df := Dataframe;"""
    wb = xw.Book(MASTER_PATH)   
    # Se leen modificaciones
    mod_cl = get_table(wb, 'ETL', [cl_col+' | '+s_col, p_col, 'Dist. Como'], cast_num=False)
    copias = [df]
    for idx, row in mod_cl.iterrows():
        nue, orig = row.dropna()
        if re.search('\d[\s,\-|]+\d', str(orig)):
            c, s = [int(n) for n in re.split('\D', orig) if n.isdigit()]
            n_c, n_s = [int(n) for n in re.split('\D', nue) if n.isdigit()]
            _ = df.loc[(df[cl_col] == c) & (df[s_col] == s), :].copy()
            _.loc[:, [cl_col, s_col]] = [n_c, n_s]
        else:
            _ = df[df[p_col] == float(orig)].copy()
            _[p_col] = float(nue)
        copias.append(_)

    return pd.concat(copias)

In [53]:
def actualizar_base():
    """Lee Historico y devuelve punto de partida (warm start) para optimización. 
    Si se indica ignorar_cl_m, se ignoran los clientes en el maestro sin distribución previa. 
    Si no, se eleva un error para indicar el ingreso de 'Dist. Como'"""
    t0 = time()
    Q0 = pd.read_csv(TMP_PATH + 'Historico.csv', sep=';')

    Q0 = distribuir_como(Q0)
    
    if Q0[Q0.duplicated(subset=[cl_col, s_col, p_col])].shape[0] > 0 : 
        print('Advertencia, hay en "Dist. Como" un producto a distribuir que ya existía en la distribución. \n', Q0[Q0.duplicated(subset=[cl_col, s_col, p_col])][p_col].unique())
    
    # eliminar repetidos?
    Q0 = Q0.groupby([cl_col, s_col, p_col]) \
            .agg({neto_col: 'sum'}) \
            .reset_index()

    # Quitar productos que no estan en presup
    restr, errs, PRESUP = get_restricciones()
    Q0 = Q0[Q0[p_col].isin(PRESUP.index)]
    # Normalizacion sobre total prod
    Q0[neto_col+'_w'] = Q0.groupby(p_col)[neto_col].apply(lambda x: x / x.sum())
    # Participacion pasada x PRESUP da el warm start
    Q0[neto_col+'_w'] = Q0.groupby(p_col)[neto_col+'_w'].apply(lambda x: x * PRESUP[x.name])


    wb = xw.Book(MASTER_PATH)
    m = get_table(wb, 'Maestro', [cl_col, s_col, c_col, z_col, nom_cli_col, cir_col], cast_num=False)

    tabla_cl = get_table(wb, 'Tabla Clientes', [c_col, ge_col, cl_col], cast_num=False)
    tabla_pr = get_table(wb, 'Tabla Productos', [p_col, 'Nombre '+ p_col, fl_col, gr_col], cast_num=False)

    d = defaultdict(LabelEncoder)
    tabla_cl[[ge_col, c_col]] = tabla_cl[[ge_col, c_col]].apply(lambda x: d[x.name].fit_transform(x))
    tabla_pr[[fl_col]] = tabla_pr[[fl_col]].apply(lambda x: d[x.name].fit_transform(x))
    tabla_pr[p_col] = pd.to_numeric(tabla_pr[p_col], downcast='integer')

    Q0[p_col] = pd.to_numeric(Q0[p_col])
    
    d = {k:list(v.classes_) for k,v in d.items()}
    d['Canal'].append('Default')
    default = len(d['Canal'])
    # output codigos
    _ = pd.DataFrame.from_dict(d, orient='index').T
    _.index.name = 'Codigo'
    wb.sheets['Codigos'].range('A1').options(pd.DataFrame, index=True).value = _
    del _

    # data cl a maestro
    m = pd.merge(tabla_cl, m, how='right', on=[cl_col], suffixes=('','_0'))
    m = m.loc[:, ~m.columns.str.endswith('_0')]
    m = m.fillna(default)

    #Maestro. Se guardan los ultimos
    wb.sheets['Maestro'].range('A2', 'T200000').clear()
    wb.sheets['Maestro'].range('A1').options(pd.DataFrame, index=False).value = m[[c_col, s_col, cl_col, nom_cli_col, cir_col, ge_col]]
        
    Q0 = pd.merge(tabla_pr.loc[:,[p_col, fl_col]], Q0, how='right', on=[p_col], suffixes=('','_0'))
    Q0 = Q0.loc[:, ~Q0.columns.str.endswith('_0')]

    # data cl maestro a q0
    Q0 = pd.merge(Q0, m, how='right', on=[cl_col, s_col], suffixes=('_0',''))
    Q0 = Q0.loc[:, ~Q0.columns.str.endswith('_0')]

    if PRESUP[~PRESUP.index.isin(tabla_pr[p_col])].shape[0] > 0: 
        raise Exception('Falta información en la tabla para los siguientes productos', PRESUP[~PRESUP.index.isin(tabla_pr[p_col])].values)
    if not Q0[Q0.isnull().any(axis=1)].shape[0] == 0: 
        print("Falta la distribución de productos para Cliente, Sucursal", Q0[Q0.isnull().any(axis=1)][[cl_col, s_col]].astype(int).values.tolist())
        Q0 = Q0.dropna(axis=0)
    if not Q0[cl_col].isin(m[cl_col]).all(): raise Exception("Se encontraron sin distribución de productos los clientes, sucursales:", Q0[Q0.isnull().any(axis=1)][cl_col].astype(int).values.tolist())
    if not Q0[p_col].isin(PRESUP.index).all(): raise Exception( "Error. Productos fuera de presupuesto.", Q0[~Q0[p_col].isin(PRESUP.index)].astype(int).values.tolist())

    assert Q0[Q0.isnull().any(axis=1)].shape[0] == 0

    Q0.drop(nom_cli_col, axis=1).astype('float64').to_csv(TMP_PATH + 'Q1.csv', sep = ';', index=False)
    ctypes.windll.user32.MessageBoxW(0, f"Base modificada. {time() - t0:.0f} segundos.",  "Terminado", 1)
    return Q0

In [54]:
def get_restricciones():
    """Toma tabla de restricciones."""
    
    wb = xw.Book(MASTER_PATH)
    restr = get_table(wb, 'Restricciones', 'Zona, C.MHSA, Canal, Grupo Economico, Cliente, Suc., Flia., Prod., Simb., Cantidad, Porcentaje del Total, Porcentaje Relativo'.split(', '), cast_num=False)
    restr['Simb.'].replace({'<=':1, '.=':0, np.nan:0}, inplace=True)

    # Extrae las filas del presupuesto (filas SOLO con producto - cantidad)
    restr = restr.apply(lambda x: pd.to_numeric(x, errors= 'coerce', downcast='integer'))
    PRESUP = restr[restr.loc[:, restr.columns[~restr.columns.isin(['Cantidad', p_col, 'Simb.'])]]
                   .isnull().all(axis = 1)].set_index(p_col)['Cantidad'].sort_index()
    PRESUP = PRESUP[(PRESUP.index.notna()) & (PRESUP > 0)]

    valid_left = restr.iloc[:, :8].notna().any(axis=1)
    valid_right = restr.iloc[:, [9, 10, 11]].notna().any(axis=1)
    errs = restr[~(valid_left & valid_right)].index
    if len(errs) > 0: print(f'Errores en restricciones con indices {errs.values + 2}')
    restr = restr[valid_left&valid_right]
    
    return restr, errs.to_series(), PRESUP

In [55]:
def prepare_vars():
    """Toma Q1.csv y las restricciones y prepara matrices para optimización."""
    Q = pd.read_csv(TMP_PATH + 'Q1.csv', sep=';')
    
    x = Q[neto_col+'_w'].values
    np.savetxt(TMP_PATH + r'\x.gz', x)
    #del x
    
    #Q = Q.iloc[:,:-2]
    pr_fl = Q.groupby(p_col)[fl_col].first()
    
    restr, errs, PRESUP = get_restricciones()
    
    A = []
    b = []
    
    for i, row in restr.iterrows():
        cols = row.iloc[:-4].dropna()
        mask = np.logical_and.reduce([(Q[c] == v) for c,v in cols.items()])
        A.append(mask)

        row = row.dropna()

        if 'Cantidad' in row.index:
            if ((fl_col in row.index) | (p_col in row.index)):
                b.append(row['Cantidad'])
            else:
                b.append(PRESUP.sum())
        elif 'Porcentaje Relativo' in row.index:
            uni = (row['Porcentaje Relativo']/100) * (Q[neto_col+'_w'].values @ mask)
            b.append(uni)
        elif 'Porcentaje del Total' in row.index:
            if (fl_col in row.index):
                uni = (row['Porcentaje del Total']/100) * PRESUP.loc[pr_fl[pr_fl == row[fl_col]].index.values].sum()
            elif p_col in row.index:
                uni = (row['Porcentaje del Total']/100) * PRESUP[pr_fl[pr_fl.index == row[p_col]].index.values].sum()
            else:
                uni = (row['Porcentaje del Total']/100) * PRESUP.sum()
            b.append(uni)
        else:
            raise Exception("##")

    idx_ineqs = restr[restr.iloc[:,-3] == 1].index.values.astype('int32')
    A = csr_matrix(np.vstack(A).astype('bool'))
    b = np.array(b).astype('int64')
    save_npz(TMP_PATH + r'\A.npz', A)
    np.savetxt(TMP_PATH + r'\b.gz', b)
    np.savetxt(TMP_PATH + r'\idx_ineqs.gz', idx_ineqs)
    
    return A, x, b, idx_ineqs

In [56]:
def check_constraints(margen=10, q = None):
    print('Ingresando resultados en Restricciones.')
    A_, q_, b_ = [TMP_PATH + p for p in ['A.npz', 'x.gz', 'b.gz']]
    A = load_npz(A_)
    if type(q) == type(None):
        q = np.loadtxt(q_)
    b = np.loadtxt(b_)

    wb = xw.Book(MASTER_PATH)
    restr = get_table(wb, 'Restricciones', 'Zona, C.MHSA, Canal, Grupo Economico, Cliente, Suc., Flia., Prod., Simb., Cantidad, Porcentaje del Total, Porcentaje Relativo'.split(', '), cast_num = False)
    restr, inval, PRESUP = get_restricciones()    
    
    res = []
    margen = 100
    for i in range(A.shape[0]):
        active = q[A[i].toarray().ravel()]
        # total para restriccion
        s = active.sum().astype(int)
        # satisfecho bool para restriccion
        r = s <= b[i] + margen 
        resto = s - b[i] 
        res.append((str(r), str(b[i]), str(s), str(round(resto, 0)), str(round((s/b[i]) * 100, 1))))

    for n in inval:
        res.insert(n, ('NA','NA','NA', 'NA', 'NA'))  
    
    wb = xw.Book(MASTER_PATH)
    wb.sheets['Restricciones'].range('P2').value = res

In [57]:
def validar(Q = None):
    t0 = time()
    print('Volcando resultado en Análisis.')
    import matplotlib.pyplot as plt
    import seaborn as sns

    wb = xw.Book(MASTER_PATH)
    filtro = get_table(wb, 'Filtro', [z_col, c_col, cl_col, s_col, cir_col, fl_col, p_col])
    filtro.dropna(how='all', inplace=True)
    if Q is None:
        Q = pd.read_csv(TMP_PATH+'Q1.csv', sep=';')

    data, fig = check_sol(Q=Q, show=True, th1=0.7, th2=0.3)
    check_constraints(q = Q[neto_col+'_1'].values)    
    
    if filtro.shape[0] == 0:
        mask = np.ones(Q.shape[0]).astype(bool)
    else:
        masks = []
        for i, row in filtro.iterrows():
            row = row.dropna()
            mask_ = np.logical_and.reduce([(Q[c] == v) for c, v in row.items()])
            masks.append(mask_)
        mask = np.logical_or.reduce(masks)
    
    Q = Q.loc[mask, :]

    wb.sheets['Visualización'].pictures.add(fig, name='Dist. de diferencias', update=True)
    wb.sheets['Visualización'].range('C4').options(transpose=False).value = {k:v for k,v in data.items() if k not in ['Negativos']}
    
    wb.sheets['Análisis'].range('A2', 'T100000').clear()
    wb.sheets['Análisis'].range('A1').options(pd.DataFrame).value = Q.groupby([cl_col, s_col]).agg({neto_col:'sum', neto_col+'_w':'sum',neto_col+'_1':'sum'}).round(3)
    wb.sheets['Análisis'].range('H1').options(pd.DataFrame).value = Q.groupby([c_col, fl_col]).agg({neto_col:'sum', neto_col+'_w':'sum',neto_col+'_1':'sum'}).round(3)
    wb.sheets['Análisis'].range('O1').options(pd.DataFrame).value = Q.groupby([cir_col, fl_col]).agg({neto_col:'sum', neto_col+'_w':'sum',neto_col+'_1':'sum'}).round(3)
    
    wb.sheets['Análisis'].range('F2').options(transpose=True).value = [f"=ROUND((E{c}-D{c})/D{c}*100, 2)" for c in range(2, Q.groupby([cl_col, s_col]).count().shape[0]+2)]
    wb.sheets['Análisis'].range('M2').options(transpose=True).value = [f"=ROUND((L{c}-K{c})/K{c}*100, 2)" for c in range(2, Q.groupby([c_col, fl_col]).count().shape[0]+2)]
    wb.sheets['Análisis'].range('T2').options(transpose=True).value = [f"=ROUND((S{c}-R{c})/R{c}*100, 2)" for c in range(2, Q.groupby([cir_col, fl_col]).count().shape[0]+2)]
    
    ctypes.windll.user32.MessageBoxW(0, f"Análisis terminado. {time() - t0:.0f} segundos.",  "Terminado", 1)

In [58]:
def solve_nras(A, q, b, idx_ineqs, precision=10, verbose=False):
    c = 0
    d_ = np.inf
    while True:
        p = (b / (A @ q))
        idx_ineqs_ok = np.where(p[idx_ineqs] > 1 + 1e-10)
        p[idx_ineqs_ok] = 1
        A_= A.multiply(p[:, np.newaxis])
        P = np.squeeze(np.array(np.true_divide(A_.sum(0), (A_!=0).sum(0))))
        q = q * P
        d = np.linalg.norm((b - (A @ q))[[idx for idx in range(A.shape[0]) if idx not in idx_ineqs_ok]])
        if verbose: print(d, end='\t')
        if d >= d_:
            c += 1
        d_ = d
        if (c == 10) | (d_ < precision):
            break
    return q

In [59]:
def solve_ultra(A, q0, b, idx_ineqs, th_max=1, th_min=0.3, max_iters=10, min_coef=2, max_coef=5,  margin=10, pt=1, pc=2, max_tries=3, shuffle=True):
    """Resuelve resolviendo mediante norma 2 y luego iterando hacia norma inf"""
    #indices de igualdades
    eqs = [n for n in range(b.shape[0]) if n not in idx_ineqs.astype(int)]
    #resultados actuales
    b_ = A @ q0
    # Se les asigna a las inecuaciones incumplidas el valor maximo permitido
    idx_ineqs_ok = np.intersect1d(np.where((b - b_) > 0), idx_ineqs).astype(int)
    b[idx_ineqs_ok] = b_[idx_ineqs_ok]
    
    #least squares
    sol, *_ = lsqr(A, b)
    #de var a subesp
    o = q0 - sol
    W = diags(q0).tocsr()
    
    thresh = np.linspace(th_min, th_max, max_iters)**pt
    coefs = np.linspace(min_coef**(1/2), max_coef**(1/2), max_iters)**pc
    res, fig = check_sol(A, q0, b, show=False)
    t = 0
    i = 0
    while True:
        i += 1
        if i == max_tries+1:
            break
        print(f"Thresh {thresh[-i]:.2f} Coef {coefs[-i]:.2f} { {k:round(v, 2) for k,v in res.items()} } ")
        ort = orth((A @ W).T.todense())
        base = W @ ort
        x = o - base @ (base.T @ (W.power(-2) @ o))
        q_ = x + sol
        res_, fig_ = check_sol(A, q_, b, i=i, t=t, show=False, th1=th_max, th2=th_min)
        if not ((res_['Max'] < res['Max']) or (res_['Min'] > res['Min']) or (res_['Norma Aq-b'] < res['Norma Aq-b'] - margin)):
            if (t == 1) and shuffle:
                np.random.shuffle(thresh)
                np.random.shuffle(coefs)
            elif t == max_tries:
                print('Max tries reached.')
                break
            else:
                t += 1
                continue
        q = q_
        res = res_    
        idx = np.argwhere(np.abs(W.power(-1) @ (q_ - q0)) > thresh[-i]).flatten().astype(int)
        if idx.shape[0] == 0:
            continue
        W[idx, idx] = W[idx, idx] * (1 / coefs[-i]) 
    return q, res

In [79]:
def check_sol(A=None, q = None, b=None, Q = None, i = 0, t = 0, show=True, th1=0.60, th2=0.3, xlim=(-1, 1), ylim=None):
    if Q is None:
        Q = pd.read_csv(TMP_PATH + 'Q1.csv', sep=';')
    if q is None:
        q = Q[neto_col+'_1']
    if A is None:
        A,_,b,idx_ineqs = prepare_vars()
        del _
    data = (np.linalg.norm(Q['Dif'], ord=2), np.linalg.norm(A @ q - b), Q[Q[neto_col+'_1'] <= 0].shape[0], *Q[neto_col+'_1'].summary().values[1:])
    names = f'Norma q-q0, Norma Aq-b, <= a 0, Media, Desvío, Mínimo, 1er Cuartil, Mediana, 3er Cuartil, Máximo'.split(', ')
    data = {k:v for k,v in zip(names, data)}
    if show:
        import matplotlib.pyplot as plt
        import seaborn as sns
        sns.set()
        show = plt.figure()
        sns.distplot(Q['Dif'], kde=False, norm_hist=True)
        lim = max(np.abs([data['Max'], data['Min']])) + 0.1
        plt.xlim(-lim, lim)  
        if ylim: plt.ylim((0, ylim))
        data = {t[0]: float(re.search(r"[\d\-]+\.*0*[.1-9]{0,2}", str(t[1])).group(0)) for t in data.items()}

    return data, show

In [36]:
def solve(polish=False):
    t0 = time()
    print('Preparando optimización.')
    A, q0, b, idx_ineqs = prepare_vars()
    sol = solve_nras(A,q0.copy(),b,idx_ineqs)
    Q = pd.read_csv(TMP_PATH+'Q1.csv', sep=';')
    Q[neto_col + '_1'] = sol
    Q['Dif'] = (1 - (Q[neto_col+'_1'] / Q[neto_col+'_w']))
    Q.to_csv(TMP_PATH + 'Q1.csv', sep=';', index=False)
    validar(Q=Q)
    if polish:
        sol, data = solve_ultra(A, sol, b, idx_ineqs, max_tries=2)
        validar()
    ctypes.windll.user32.MessageBoxW(0, f"Distribución calculada. {time() - t0:.0f} segundos.",  "Terminado", 1)